In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
import pandas as pd
import numpy as np
from PIL import Image
from sklearn.model_selection import train_test_split
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D, Flatten, Dense

In [15]:
data_benign = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/Grayscale-Image/staDynBenignLab.csv')
data_mal1 = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/Grayscale-Image/staDynVxHeaven2698Lab.csv')
data_mal2 = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/Grayscale-Image/staDynVt2955Lab.csv')

In [16]:
benign_filenames = data_benign['filename'].values
mal2_filenames = data_mal2['filename'].values

# Load the grayscale images and labels
image_filenames = []
labels = []

# Benign (staDynBenignLab.csv)
for i in range(len(data_benign['filename'])):
  file_name = benign_filenames[i]
  image_filenames.append(f'/content/drive/MyDrive/Colab Notebooks/Grayscale-Image/Benign-Images/{file_name}.png')
  labels.append(data_benign['label'][i])

# Malware 1 (staDynVxHeaven2698Lab.csv)
for i in range(len(data_mal1['label'])):
  image_filenames.append(f'/content/drive/MyDrive/Colab Notebooks/Grayscale-Image/Malware-Images/staDynVxHeaven2698Lab/{i}_VxHeaven_malware.png')
  labels.append(data_mal1['label'][i])

# Malware 2 (staDynVt2955Lab.csv)
for i in range(len(data_benign['filename'])):
  file_name = mal2_filenames[i]
  image_filenames.append(f'/content/drive/MyDrive/Colab Notebooks/Grayscale-Image/Malware-Images/staDynVt2955Lab/{file_name}.png')
  labels.append(data_mal2['label'][i])

In [17]:
# Load and preprocess the images
images = []
for filename in image_filenames:
    img = Image.open(filename).convert('L')
    img_array = np.array(img)
    images.append(img_array)

images = np.array(images)
labels = np.array(labels)

In [18]:
# Split the data into training and testing sets
x_train, x_test, y_train, y_test = train_test_split(images, labels, test_size=0.2, random_state=42)

In [19]:
# Normalize the image data
x_train = x_train / 255.0
x_test = x_test / 255.0

# Reshape the data for CNN input
x_train = x_train.reshape(-1, 33, 33, 1)
x_test = x_test.reshape(-1, 33, 33, 1)

# Convert labels to one-hot encoding
num_classes = 2
y_train = np.eye(num_classes)[y_train]
y_test = np.eye(num_classes)[y_test]

In [20]:
# Build the CNN model
model = Sequential()
model.add(Conv2D(32, kernel_size=(3, 3), activation='relu', input_shape=(33, 33, 1)))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Conv2D(64, kernel_size=(3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Flatten())
model.add(Dense(128, activation='relu'))
model.add(Dense(num_classes, activation='softmax'))

In [21]:
# Compile the model
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

# Train the model
model.fit(x_train, y_train, batch_size=32, epochs=10, validation_data=(x_test, y_test))

Epoch 1/10
98/98 [==============================] - 5s 39ms/step - loss: 0.4430 - accuracy: 0.8453 - val_loss: 0.4755 - val_accuracy: 0.8188
Epoch 2/10
98/98 [==============================] - 2s 25ms/step - loss: 0.3962 - accuracy: 0.8540 - val_loss: 0.4488 - val_accuracy: 0.8188
Epoch 3/10
98/98 [==============================] - 2s 25ms/step - loss: 0.3521 - accuracy: 0.8543 - val_loss: 0.3809 - val_accuracy: 0.8188
Epoch 4/10
98/98 [==============================] - 2s 25ms/step - loss: 0.3184 - accuracy: 0.8576 - val_loss: 0.3876 - val_accuracy: 0.8201
Epoch 5/10
98/98 [==============================] - 2s 25ms/step - loss: 0.3028 - accuracy: 0.8669 - val_loss: 0.3324 - val_accuracy: 0.8496
Epoch 6/10
98/98 [==============================] - 4s 38ms/step - loss: 0.3009 - accuracy: 0.8666 - val_loss: 0.3367 - val_accuracy: 0.8406
Epoch 7/10
98/98 [==============================] - 2s 25ms/step - loss: 0.3087 - accuracy: 0.8643 - val_loss: 0.3415 - val_accuracy: 0.8355
Epoch 8/10
98

In [22]:
# Evaluate the model
loss, accuracy = model.evaluate(x_test, y_test)
print('Test Loss:', loss)
print('Test Accuracy:', accuracy)

25/25 [==============================] - 0s 7ms/step - loss: 0.3390 - accuracy: 0.8303
Test Loss: 0.338995099067688
Test Accuracy: 0.8303341865539551
